<a href="https://colab.research.google.com/github/bhaveshsingh0206/Question-Answering/blob/main/sQuAd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import json
# import pandas as pd
# dataset = pd.DataFrame()
# dataset["context"] = ""
# dataset["question"] = ""
# dataset["answer"] = ""
# dataset["id"] = ""
# with open('/content/drive/My Drive/train.json', 'r') as f:
#   data = json.loads(f.read())
# count = 0
# for i in data["data"]:
#   for k in range(len(i["paragraphs"])):
#     for j in i["paragraphs"][k]["qas"]: 
#       dataset.loc[count, "question"] = j["question"]
#       if j["is_impossible"]:
#         dataset.loc[count, "answer"] = j["plausible_answers"][0]["text"]
#       else:
#         dataset.loc[count, "answer"] = j["answers"][0]["text"]
#       dataset.loc[count, "id"] = j["id"]
#       dataset.loc[count, "context"] = i["paragraphs"][k]["context"]
#       count += 1

In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/My Drive/question-answer.csv')
dataset.head()

,context,question,answer,id
0,"In Serbia, the Serbian language is the officia...",In which province are Serbian and Croatian bot...,Vojvodina,572a12b41d04691400779735
1,"Imperial submitted a total of 1,257 staff acro...",What was Imperial's overall GPA rank according...,2nd,570a4eb44103511400d595fc
2,"On September 30, 1989, thousands of Belorussia...",When did the Chernobyl disaster happen?,1986,572827feff5b5019007d9e3c
3,Each code point has a single General Category ...,What is the title of the Separator category in...,"Other, Control",5acd126d07355d001abf334f
4,Melatonin is absent from the system or undetec...,What is more stable than the melatonin offset?,core temperature minimum,5a223cec819328001af38a49


In [ ]:
dataset.context[0]

'In Serbia, the Serbian language is the official one, while both Serbian and Croatian are official in the province of Vojvodina. A large Bosniak minority is present in the southwest region of Sandžak, but the "official recognition" of Bosnian language is moot. Bosnian is an optional course in 1st and 2nd grade of the elementary school, while it is also in official use in the municipality of Novi Pazar. However, its nomenclature is controversial, as there is incentive that it is referred to as "Bosniak" (bošnjački) rather than "Bosnian" (bosanski) (see Bosnian language for details).'

In [ ]:
dataset.question[10]

'How many litres of water will the Wonthaggi desalination plant be capbale of treating per year?'

In [ ]:
dataset.answer[10]

'150 billion litres'

In [ ]:
# dataset.context[200]
# dataset.to_csv('/content/drive/My Drive/question-answering.csv', index=False)

In [ ]:
# word2Vec = {}
# with open('/content/drive/My Drive/glove.6B.300d.txt') as f:
#   for line in f:
#     line = line.rstrip().split(" ")
#     word = line[0]
#     wordVector = line[1:]
#     if word not in word2Vec:
#       word2Vec[word] = wordVector

In [ ]:
# import numpy as np
# np.save('/content/drive/My Drive/word2Vec.npy', word2Vec) 
# import json

# Serialize data into file:
# json.dump( word2Vec, open( "/content/drive/My Drive/word2Vec.json", 'w' ) )

# Read data from file:
# word2Vec = json.load( open( "/content/drive/My Drive/word2Vec.json" ) )
# Load
# read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()
# print(read_dictionary['hello'])

In [ ]:
dataset.fillna("No idea", inplace = True)

In [ ]:
# dataset = dataset.sample(frac=1).reset_index(drop=True)
# dataset.to_csv('question-answer.csv', index=False)

In [ ]:
text = dataset["context"][100000]
# text = "hi's my, [][] name is any.fv53w43q===="
text

"Somerset is an important supplier of defence equipment and technology. A Royal Ordnance Factory, ROF Bridgwater was built at the start of the Second World War, between the villages of Puriton and Woolavington, to manufacture explosives. The site was decommissioned and closed in July 2008. Templecombe has Thales Underwater Systems, and Taunton presently has the United Kingdom Hydrographic Office and Avimo, which became part of Thales Optics. It has been announced twice, in 2006 and 2007, that manufacturing is to end at Thales Optics' Taunton site, but the trade unions and Taunton Deane District Council are working to reverse or mitigate these decisions. Other high-technology companies include the optics company Gooch and Housego, at Ilminster. There are Ministry of Defence offices in Bath, and Norton Fitzwarren is the home of 40 Commando Royal Marines. The Royal Naval Air Station in Yeovilton, is one of Britain's two active Fleet Air Arm bases and is home to the Royal Navy's Lynx helic

In [ ]:
SOS_TOKEN = '<start>'
EOS_TOKEN = '<end>'
PAD_TOKEN = '<pad>'
UNK_TOKEN = '<unk>'

SOS_idx = 0
EOS_idx = 1
PAD_idx = 2
UNK_idx = 3

In [ ]:
class Corpus:
  def __init__(self):
    self.vocab_size = 4
    self.word2idx = {
        SOS_TOKEN: SOS_idx,
        EOS_TOKEN: EOS_idx,
        PAD_TOKEN: PAD_idx,
        UNK_TOKEN: UNK_idx
    }
    self.extended = 0
    self.idx2word = {k:word for word,k in self.word2idx.items()}

  def readSentence(self, sent):
    for word in sent:
      self.addWordToDictionary(word)
  
  def addWordToDictionary(self, word):
    if word not in self.word2idx:
      self.word2idx[word] = self.vocab_size
      self.idx2word[self.vocab_size] = word
      self.vocab_size += 1
  
  def checkForExtraVocab(self, vocab):
    for i in vocab.word2idx:
      if i not in self.word2idx:
        self.extended += 1
        self.addWordToDictionary(i)
  
  def indices2words(self, indices):
    return [self.idx2word[indice] for indice in indices]


In [ ]:
import string
from tqdm import tqdm
def prepareCorpus(sequence):
  freq_words = {}
  sents_tokenized = []
  for i,sent in tqdm(enumerate(sequence)):
    
    words = sent.lower().split()
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]
    sent_tokenized = [w for w in stripped if len(w) != 0]
    
    sents_tokenized.append(sent_tokenized)
  return sents_tokenized

In [ ]:
context_tokenized = prepareCorpus(dataset['context'])
question_tokenized = prepareCorpus(dataset['question'])
answer_tokenized = prepareCorpus(dataset['answer'])

130319it [00:11, 11821.20it/s]
130319it [00:01, 84124.16it/s]
130319it [00:01, 124208.07it/s]


In [ ]:
len(context_vocab.idx2word)

106133

In [ ]:
def createVocab(sequences):
  corpus = Corpus()
  for sent in sequences:
    corpus.readSentence(sent)
  return corpus

In [ ]:
question_vocab = createVocab(question_tokenized)
context_vocab = createVocab(context_tokenized+question_tokenized+answer_tokenized)
answer_vocab = createVocab(answer_tokenized)

In [ ]:
["ok"] + ["mo"] + ["po"]

['ok', 'mo', 'po']

In [ ]:
answer_tokenized[0]

['vojvodina']

In [ ]:
import numpy as np

In [ ]:
len(context_vocab.word2idx)
# len(answer_vocab.word2idx)
# # Create one common vocab:
# context_vocab.checkForExtraVocab(question_vocab)
# len(context_vocab.word2idx)
# context_vocab.checkForExtraVocab(answer_vocab)
# len(context_vocab.word2idx)

106133

In [ ]:
print(context_tokenized[0])

['in', 'serbia', 'the', 'serbian', 'language', 'is', 'the', 'official', 'one', 'while', 'both', 'serbian', 'and', 'croatian', 'are', 'official', 'in', 'the', 'province', 'of', 'vojvodina', 'a', 'large', 'bosniak', 'minority', 'is', 'present', 'in', 'the', 'southwest', 'region', 'of', 'sandžak', 'but', 'the', 'official', 'recognition', 'of', 'bosnian', 'language', 'is', 'moot', 'bosnian', 'is', 'an', 'optional', 'course', 'in', '1st', 'and', '2nd', 'grade', 'of', 'the', 'elementary', 'school', 'while', 'it', 'is', 'also', 'in', 'official', 'use', 'in', 'the', 'municipality', 'of', 'novi', 'pazar', 'however', 'its', 'nomenclature', 'is', 'controversial', 'as', 'there', 'is', 'incentive', 'that', 'it', 'is', 'referred', 'to', 'as', 'bosniak', 'bošnjački', 'rather', 'than', 'bosnian', 'bosanski', 'see', 'bosnian', 'language', 'for', 'details']


In [ ]:
context_token = []
question_token = []
answer_token = []
for index, sent in enumerate(context_tokenized):
  if len(sent) <= 150:
    context_token.append(sent)
    question_token.append(question_tokenized[index])
    answer_token.append(answer_tokenized[index])

In [ ]:
context_tokenized = context_token
question_tokenized = question_token
answer_tokenized = answer_token

In [ ]:
DIMENSION = 300
content_max_length = max(len(s) for s in context_tokenized)
question_max_length = max(len(s) for s in question_tokenized)
answer_max_length = max(len(s) for s in answer_tokenized) + 2
MAX_VOCAB = len(context_vocab.word2idx)

In [ ]:
len(context_tokenized)

103767

In [ ]:
# from tqdm import tqdm
# word2Vec = {}
# with open('/content/drive/My Drive/Copy of glove.6B.300d.txt') as f:
#   for line in tqdm(f):
#     line = line.rstrip().split(' ')
#     word = line[0]
#     embedding_vector = line[1:]
#     word2Vec[word] = embedding_vector

In [ ]:
# print(len(word2Vec))

In [ ]:
# print(word2Vec["person"])

In [ ]:
# embedding_matrix[90][0]

In [ ]:
import torch
# import numpy as np
# out_of_vocab = []
# target_embedding_matrix = np.zeros((MAX_VOCAB, DIMENSION))
# for j,k in context_vocab.word2idx.items():
#   if k < MAX_VOCAB:
#     embedding_vector = word2Vec.get(j)
#     if embedding_vector is not None:
#       target_embedding_matrix[k] = embedding_vector
#     else:
#       out_of_vocab.append(j)


# embedding_matrix_name = 'embedding_matrix.pt'
# path = f"/content/drive/My Drive/{embedding_matrix_name}" 
# torch.save(target_embedding_matrix, path)
embedding_matrix = torch.load('/content/drive/My Drive/embedding_matrix.pt')

In [ ]:
# len(out_of_vocab)

In [ ]:
RANDOM_SEED = 10
np.random.seed(RANDOM_SEED)

DATASET_LENGTH = len(context_tokenized)
permuted_indices = np.random.permutation(DATASET_LENGTH)

training_context  = [context_tokenized[i] for i in permuted_indices[:85000]]
training_question = [question_tokenized[i] for i in permuted_indices[:85000]]
training_answer   = [answer_tokenized[i] for i in permuted_indices[:85000]]


validation_context  = [context_tokenized[i] for i in permuted_indices[85000:100000]]
validation_question = [question_tokenized[i] for i in permuted_indices[85000:100000]]
validation_answer   = [answer_tokenized[i] for i in permuted_indices[85000:100000]]

test_context  = [context_tokenized[i] for i in permuted_indices[100000:]]
test_question = [question_tokenized[i] for i in permuted_indices[100000:]]
test_answer   = [answer_tokenized[i] for i in permuted_indices[100000:]]

In [ ]:
import torch
def indexedFromSentence(vocab, sent):
  return [vocab.word2idx[word] for word in sent]

def tensorFromSentence(vocab, sent, max_seq_len, pad):
  transformed = indexedFromSentence(vocab, sent)
  if pad:  
    transformed.append(EOS_idx)
    transformed.insert(0, SOS_idx)
  if len(transformed) < max_seq_len:
    transformed += [PAD_idx] * (max_seq_len - len(transformed))
  return torch.LongTensor(transformed)

def tensorFromPairs(context, question, answer, content_max_length, question_max_length, answer_max_length):
  context_tensor = tensorFromSentence(context_vocab, context, content_max_length, False).unsqueeze(1)
  question_tensor = tensorFromSentence(context_vocab, question, question_max_length, False).unsqueeze(1)
  answer_tensor = tensorFromSentence(context_vocab, answer, answer_max_length, True).unsqueeze(1)
  return (context_tensor, question_tensor, answer_tensor)

In [ ]:
training_set = []
for context, question, answer in zip(training_context, training_question, training_answer):
  training_set.append(tensorFromPairs(context, question, answer, content_max_length, question_max_length, answer_max_length))

validation_set = []
for context, question, answer in zip(validation_context, validation_question, validation_answer):
  validation_set.append(tensorFromPairs(context, question, answer, content_max_length, question_max_length, answer_max_length))

test_set = []
for context, question, answer in zip(test_context, test_question, test_answer):
  test_set.append(tensorFromPairs(context, question, answer, content_max_length, question_max_length, answer_max_length))

In [ ]:
X_context_train, X_question_train, y_train = zip(*training_set)
X_context_validation, X_question_validation, y_validation = zip(*validation_set)
X_context_test, X_question_test, y_test = zip(*test_set)

In [ ]:
X_context_train = torch.transpose(torch.cat(X_context_train, dim=-1), 1, 0)
X_question_train = torch.transpose(torch.cat(X_question_train, dim=-1), 1, 0)
y_train = torch.transpose(torch.cat(y_train, dim=-1), 1, 0)

X_context_validation = torch.transpose(torch.cat(X_context_validation, dim=-1), 1, 0)
X_question_validation = torch.transpose(torch.cat(X_question_validation, dim=-1), 1, 0)
y_validation = torch.transpose(torch.cat(y_validation, dim=-1), 1, 0)


X_context_test = torch.transpose(torch.cat(X_context_test, dim=-1), 1, 0)
X_question_test = torch.transpose(torch.cat(X_question_test, dim=-1), 1, 0)
y_test = torch.transpose(torch.cat(y_test, dim=-1), 1, 0)

In [ ]:
import torch.nn as nn
import torch
from torch.autograd import Variable

In [ ]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], \
        requires_grad=False).to(device)
        return x

In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, max_vocab, dim, nhead, dropout, no_of_layers, context_seq_len):
    super(TransformerEncoder, self).__init__()
    self.embedding = nn.Embedding(max_vocab, dim)
    self.embedding.weights = torch.nn.Parameter(torch.from_numpy(embedding_matrix).to(device))
    self.embedding.weight.requires_grad = True
    self.positional = PositionalEncoder(d_model=dim, max_seq_len=context_seq_len)
    layers = OrderedDict()
    for i in range(no_of_layers):
      layers[str(i)] = nn.TransformerEncoderLayer(d_model=dim, nhead=nhead, dropout=dropout)
    self.E = nn.Sequential(layers)

  def forward(self, X):
    x = self.embedding(X)
    x = self.positional(x)
    # x-> batch, no_words, dimensions
    x = x.permute(1, 0, 2)
    transformer_encoder_output = self.E(x)
    return transformer_encoder_output.to(device)

In [ ]:
class TransformerDecoder(nn.Module):
  def __init__(self, max_vocab, dim, dropout, question_seq_len, nhead):
    super(TransformerDecoder, self).__init__()
    self.embedding = nn.Embedding(max_vocab, dim)
    self.embedding.weights = torch.nn.Parameter(torch.from_numpy(embedding_matrix).to(device))
    self.embedding.weight.requires_grad = True
    self.positional = PositionalEncoder(dim, question_seq_len)
    self.decoder1 = nn.TransformerDecoderLayer(d_model=dim, nhead=nhead, dropout=dropout)
    self.decoder2 = nn.TransformerDecoderLayer(d_model=dim, nhead=nhead, dropout=dropout)
    self.decoder3 = nn.TransformerDecoderLayer(d_model=dim, nhead=nhead, dropout=dropout)
    self.decoder4 = nn.TransformerDecoderLayer(d_model=dim, nhead=nhead, dropout=dropout)
    self.question_seq_len = question_seq_len
    self.T = nn.Sequential (
        nn.Linear(question_seq_len*dim,1024),
        nn.ReLU(),
        # nn.BatchNorm2d(1024),
        nn.Dropout(p=0.2),
        nn.Linear(1024, 512),
        nn.ReLU()
    )
    
  def forward(self, X, enc_out):
    # print(self.question_seq_len)
    # print("Original ", X.shape)
    batch_size = X.shape[0]
    x = self.embedding(X)
    x = self.positional(x)
    x = x.permute(1, 0, 2)
    x = self.decoder1(tgt=x, memory=enc_out)
    x = self.decoder2(tgt=x, memory=enc_out)
    x = self.decoder3(tgt=x, memory=enc_out)
    x = self.decoder4(tgt=x, memory=enc_out)
    # x-> target_words, batch, dimension
    # print(x.shape)
    x = x.permute(1, 0, 2)
    # print("Yes")
    # print(x.shape)
    # [16, 150, 300]
    x = x.reshape(batch_size, -1)
    x = self.T(x)
    x = x.unsqueeze(1)
    # x-> batch, 1, 512
    return x.to(device)

In [ ]:
class Seq2SeqDecoder(nn.Module):
  def __init__(self, max_vocab, dim, units):
    super(Seq2SeqDecoder, self).__init__()
    self.embedding = nn.Embedding(max_vocab, dim)
    self.embedding.weights = torch.nn.Parameter(torch.from_numpy(embedding_matrix).to(device))
    self.embedding.weight.requires_grad = True 
    self.gru = nn.GRU(dim, units, batch_first=True)
    self.final = nn.Linear(units, max_vocab)
    self.softmax = nn.Softmax()
  def forward(self, y, hidden_state):
    hidden_state = hidden_state.permute(1, 0, 2)
    batch_output = torch.Tensor().to(device)
    # print("hidden_state ", hidden_state.shape)
    decoder_input_single = y[:,0]
    # print("decoder_input_single", decoder_input_single.shape)
    for i in range(y.shape[1]):
      decoder_input_single = self.embedding(decoder_input_single).unsqueeze(1)
      decoder_output, hidden_state = self.gru(decoder_input_single, hidden_state)
      # print("decoder_output ", decoder_output.shape)
      decoder_output = self.final(decoder_output.squeeze(1))
      batch_output = torch.cat((batch_output, decoder_output.unsqueeze(2)), 2)
      if random.uniform(0, 1) > .5:
        decoder_input_single = y[:, i]
      else:
        decoder_input_single = decoder_output.argmax(dim=1)

    return batch_output





  def predict(self, context):
      current_y = SOS_idx
      result = [current_y]
      counter = 0
      while current_y != EOS_idx and counter<=20:
        inpu = torch.tensor([current_y]).to(device)
        # print(inpu.shape)
        inpu = inpu.unsqueeze(0)
        inpu = self.embedding(inpu)
        # print(inpu.shape)
        # batch, 1, dimension
        decoder_output, decoder_hidden_states = self.gru(inpu, context)
        # decoder_output -> batch, 1, dimension
        # print(decoder_output.shape)
        h = self.final(decoder_output.squeeze(1))
        # print(h.shape)   
        # h-> batch, mac_vocab

        y = self.softmax(h)
        current_y = torch.argmax(y, dim=1).item()
        # print(current_y.item())
      #   # print(current_y)
      #   # print(type(current_y))
        # current_y = current_y.item()
        result.append(current_y)
        counter += 1
      return result


In [ ]:
class Model(nn.Module):
  def __init__(self, max_vocab, dim, nhead, dropout, no_of_layers, question_seq_len, units, context_seq_len):
    super(Model, self).__init__()
    self.encoder = TransformerEncoder(max_vocab, dim, nhead, dropout, no_of_layers, context_seq_len)
    self.decoder = TransformerDecoder(max_vocab, dim, dropout, question_seq_len, nhead)
    self.answerdecoder = Seq2SeqDecoder(max_vocab, dim, units)
  def forward(self, context, question, answer):
    # print("started")
    enc_out = self.encoder(context)
    # print("first step done")
    seq2seqdecoderhidden = self.decoder(question, enc_out)
    # print("question")
    output = self.answerdecoder(answer, seq2seqdecoderhidden)
    # print("answer")
    return output
  
  def predict(self, context, question):
    enc_out = self.encoder(context)
    seq2seqdecoderhidden = self.decoder(question, enc_out)
    output = self.answerdecoder.predict(seq2seqdecoderhidden)
    return output

In [ ]:
import random
import math
from collections import OrderedDict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.optim import Adam
 
model = Model(max_vocab=MAX_VOCAB, dim=300, nhead=10, dropout=0.1, no_of_layers=4, question_seq_len=question_max_length, units=512, context_seq_len=content_max_length)


optim = Adam(model.parameters(), lr=0.001)
cross_entropy = nn.CrossEntropyLoss()
model.to(device)
model.load_state_dict(torch.load('/content/drive/My Drive/model/Squad Modified Epoch 10 loss: 0.5499.pt'))


<All keys matched successfully>

In [ ]:
def batch_generator(batch_indices, batch_size):
    batches = math.ceil(len(batch_indices)/batch_size)
    for i in range(batches):
        batch_start = i*batch_size
        batch_end = (i+1)*batch_size
        if batch_end > len(batch_indices):
            yield batch_indices[batch_start:]
        else:
            yield batch_indices[batch_start:batch_end]

In [ ]:
shift = torch.randperm(X_context_train.shape[0])
print(X_context_train.shape)
X_context_train = X_context_train[shift]
print(X_context_train.shape)
    # X_question_train = X_question_train[shift]
    # y_train = y_train[shift]  

torch.Size([85000, 150])
torch.Size([85000, 150])


In [ ]:
print(device)
from tqdm import tqdm_notebook as tqdm

BATCH_SIZE = 32
total_batches = int(len(X_context_train)/BATCH_SIZE) + 1
total_valid_batches = int(len(X_context_validation)/BATCH_SIZE) + 1
indices = list(range(len(X_context_train)))
valid_indices = list(range(len(X_context_validation)))

for epoch in range(3, 15):
    total_loss = 0.0
    valid_total_loss = 0.0
    torch.manual_seed(epoch)
    for step, batch in tqdm(enumerate(batch_generator(indices, BATCH_SIZE)),
                            desc='Training epoch {}'.format(epoch+1),
                            total=total_batches):
 
          x_context = X_context_train[batch, :].to(device)
          x_question = X_question_train[batch, :].to(device)
          
          
          y_tf = y_train[batch, :-1].to(device)
          y_true = y_train[batch, 1:].to(device)
          
          
          H = model.forward(x_context, x_question, y_tf)
          
          loss = cross_entropy(H, y_true)
          
          assert loss.item() > 0

          optim.zero_grad()
          loss.backward()
            
          torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
          optim.step()
          total_loss += loss.item()
    shift = torch.randperm(X_context_train.shape[0])
    X_context_train = X_context_train[shift]
    X_question_train = X_question_train[shift]
    y_train = y_train[shift]
    print('Epoch  {} training is finished, loss: {:.4f}'.format(epoch, total_loss/(step)))     
    
    
    for s, batch in tqdm(enumerate(batch_generator(valid_indices, BATCH_SIZE)),
                            desc='Training epoch {}'.format(epoch+1),
                            total=total_valid_batches):
          x_context = X_context_validation[batch, :].to(device)
          x_question = X_question_validation[batch, :].to(device)
          
          
          y_tf = y_validation[batch, :-1].to(device)
          y_true = y_validation[batch, 1:].to(device)
          
          
          H = model.forward(x_context, x_question, y_tf)
          
          loss = cross_entropy(H, y_true)
          valid_total_loss += loss.item()

    print('Epoch  {} Validation is finished, loss: {:.4f}'.format(epoch, valid_total_loss/(s)))     

    model_save_name = 'Squad Modified Epoch {} loss: {:.4f}.pt'.format(epoch, total_loss/(step))
    path = F"/content/drive/My Drive/model/{model_save_name}" 
    torch.save(model.state_dict(), path)
    print("Model saved")
    print("------------------------------------------------------------------")

cuda


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



Epoch  3 training is finished, loss: 0.5635


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch  3 Validation is finished, loss: 0.7197
Model saved
------------------------------------------------------------------


...


Epoch  11 Validation is finished, loss: 0.7394
Model saved
------------------------------------------------------------------



Epoch  12 training is finished, loss: 0.5493



Epoch  12 Validation is finished, loss: 0.7428
Model saved
------------------------------------------------------------------



Epoch  13 training is finished, loss: 0.5490



Epoch  13 Validation is finished, loss: 0.7443
Model saved
------------------------------------------------------------------


In [ ]:
for i in range(5,9):
    context = X_context_test[i,:].unsqueeze(0)
    question = X_question_test[i,:].unsqueeze(0)
    print("Context : ",' '.join(context_vocab.indices2words(context.squeeze(0).numpy())))
    print("Question : ", ' '.join(context_vocab.indices2words(question.squeeze(0).numpy())))

    y = model.predict(context.to(device), question.to(device))
    answer = ' '.join(context_vocab.indices2words(y[i:-1]))
    print("Original Answer : ",' '.join(context_vocab.indices2words(y_test[i,:].numpy())))
    print("Predicted Answer : ", ' '.join(context_vocab.indices2words(y)))